In [32]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib/"
sys.path.insert( 0, lib_dir )

In [33]:
from lib import *

In [34]:
from rauth import OAuth2Service
import requests

class AlperiaOAuth2Client:
    def __init__(self):
        self.access_token = None

        self.service = OAuth2Service(
            name="ftt01",
            client_id="47wWdaYVrvZAXumaBYpMtnkkVp2toXsmaFpB7aqjzhQFCtAZQJgMrre9jCyR7tY4",
            client_secret="Ecw2yA2fTToM4ycwrEvAhgvtKRa6PrGR3EXBCZhTyW7MUyibBhJWs2vqi8vRucpU",
            access_token_url="https://gaia-api-noprod.alperia.digital/connect/token",
            authorize_url="https://gaia-api-noprod.alperia.digital/connect/token",
            base_url="https://gaia-api-noprod.alperia.digital/",
        )

        self.get_access_token()

    def get_access_token(self):
        data = {
                'grant_type': 'client_credentials', # generally required! 
               }

        session = self.service.get_auth_session(data=data, decoder=json.loads)

        self.access_token = session.access_token

    def get_streamflow(self, basin, date_from, date_to):
        base_url = "https://gaia-api-noprod.alperia.digital/api/portate"
        params = {
            "Bacino" : basin,
            "DateFrom" : date_from,
            "DateTo" : date_to
        }
        response = requests.get( base_url, params=params, headers={'Authorization': 'Bearer {myToken}'.format(myToken=self.access_token)})
        if response.status_code == 200:
            return response.json()
        else:
            print("Error: " + str(response.status_code))
            return []
        
        response.close()

In [35]:
def save_to_csv(json_data, output_filename):
    datetime_vector = [pd.to_datetime(el['date'],utc=True) + dt.timedelta(hours=el['hour']) for el in json_data]
    values_vector = [ float(el['value']) for el in json_data ]

    df = pd.DataFrame(columns=['datetime','values'])
    df["datetime"] = datetime_vector
    df["values"] = values_vector

    try:
        old_data = pd.read_csv(output_filename)
        old_data['datetime'] = pd.to_datetime(old_data['datetime'], format='%Y-%m-%d %H:%M:%S', utc=True )
        old_data.set_index('datetime', inplace=True)

        df = append_data(old_data,df)
        df.reset_index(inplace=True)
    except:
        print('No old data!')

    df['datetime'] = [ el.strftime('%Y-%m-%dT%H:%M:%SZ%z') for el in df['datetime'] ]

    df.set_index('datetime', inplace=True)
    df = df[['values']]

    df.to_csv(output_filename)

    del df

In [36]:
pp = AlperiaOAuth2Client()

In [37]:
end_date = dt.datetime.today()

output_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/alperia/"

In [38]:
## SB002
output_filename = output_path + "SB002.csv"
##
json_data = pp.get_streamflow('Vasca Valdurna', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [39]:
## SB003
output_filename = output_path + "SB003.csv"
##
json_data = pp.get_streamflow('Vasca Corvara', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [40]:
## SB004
output_filename = output_path + "SB004.csv"
##
json_data = pp.get_streamflow('Vasca di Carico Enerpass', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [41]:
## SB005 - dati provincia ma forniti validati
output_filename = output_path + "SB005.csv"
input_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/alperia/v3/aggiornamento_dati/validati-SB005_Pontives_orari.csv"
input_data = pd.read_csv(input_data_path, skiprows=0, index_col=0, parse_dates=True)

local_tz = pytz.utc
utc_tz = pytz.utc

df = pd.DataFrame(columns=['datetime','values'])
df['datetime'] = [ lt.astimezone(utc_tz) for lt in [ local_tz.localize(t) for t in input_data.index ]]
### apply delay to the timeseries from UTC+1 to UTC
df['datetime'] = [ el-dt.timedelta(hours=1) for el in df['datetime'] ]

df['datetime'] = [ el.strftime('%Y-%m-%dT%H:%M:%SZ%z') for el in df['datetime'] ]
df.set_index('datetime', inplace=True)

vals = []
for v in input_data.values:
    try:
        act_val = round(float(v), 2)
        if act_val > 0.01:
            vals.append(act_val)
        else:
            raise
    except:
        vals.append(np.NaN)
df['values'] = vals

df.to_csv(output_filename)

In [42]:
## SB006
output_filename = output_path + "SB006.csv"
##
json_data = pp.get_streamflow('Vasca di Rio Riva', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [43]:
## SB007 - dati provincia ma forniti validati [UTC+1]
output_filename = output_path + "SB007.csv"
input_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/alperia/v3/aggiornamento_dati/validati-SB007_Trens_orari.csv"

input_data = pd.read_csv(input_data_path, skiprows=0, index_col=0, parse_dates=True)

local_tz = pytz.utc
utc_tz = pytz.utc

df = pd.DataFrame(columns=['datetime','values'])
df['datetime'] = [ lt.astimezone(utc_tz) for lt in [ local_tz.localize(t) for t in input_data.index ]]
### apply delay to the timeseries from UTC+1 to UTC
df['datetime'] = [ el-dt.timedelta(hours=1) for el in df['datetime']]

df['datetime'] = [ el.strftime('%Y-%m-%dT%H:%M:%SZ%z') for el in df['datetime']]
df.set_index('datetime', inplace=True)

vals = []
for v in input_data.values:
    try:
        act_val = round(float(v), 2)
        if act_val > 0.01:
            vals.append(act_val)
        else:
            raise
    except:
        vals.append(np.NaN)
df['values'] = vals

df.to_csv(output_filename)

In [44]:
## SB008
output_filename = output_path + "SB008.csv"
# input_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/alperia/curon.csv"
# local_tz = pytz.timezone("Europe/Rome")
# utc_tz = pytz.utc

# input_data = pd.read_csv(input_data_path, skiprows=0, header=None, index_col=0, parse_dates=True)
json_data = pp.get_streamflow('Vasca Melago', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [45]:
# df = pd.DataFrame(columns=['datetime','values'])
# df['datetime'] = [ lt.astimezone(utc_tz) for lt in [ local_tz.localize(t) for t in input_data.index ]]
# df['datetime'] = [ el.strftime('%Y-%m-%dT%H:%M:%SZ%z') for el in df['datetime']]
# df.set_index('datetime', inplace=True)

# vals = []
# for v in input_data.values:
#     try:
#         act_val = round(float(v), 2)
#         if act_val > 0.01:
#             vals.append(act_val)
#         else:
#             raise
#     except:
#         vals.append(np.NaN)
# df['values'] = vals

# df.to_csv(output_filename)

In [46]:
## SB009
output_filename = output_path + "SB009.csv"
# input_data_path = "/home/daniele/downloads/portate_castelbello_produzione.csv"
# local_tz = pytz.timezone("Europe/Rome")
# utc_tz = pytz.utc

# input_data = pd.read_csv(input_data_path, header=None, parse_dates=True, index_col=0)
# df = pd.DataFrame(columns=['datetime','values'])
# df['datetime'] = [ lt.astimezone(utc_tz) for lt in [ local_tz.localize(t) for t in input_data.index ]]
# df['datetime'] = [ el.strftime('%Y-%m-%dT%H:%M:%SZ%z') for el in df['datetime']]
# df.set_index('datetime', inplace=True)

# vals = []
# for v in input_data.values:
#     try:
#         vals.append(round(float(v), 2))
#     except:
#         vals.append(np.NaN)
# df['values'] = vals

# df.to_csv(output_filename)
json_data = pp.get_streamflow('Vasca Castelbello', '1900-01-01 00:00', end_date.strftime('%Y-%m-%d %H:%M'))
save_to_csv(json_data, output_filename)

No old data!


In [47]:
# local_tz = pytz.timezone("Europe/Rome")

# start_date = dt.datetime( 2021, 8, 15, tzinfo=local_tz )
# end_date = dt.datetime( 2021, 12, 12, tzinfo=local_tz )
# # end_date = dt.datetime.today().replace(tzinfo=local_tz) + dt.timedelta( days=2 )

In [48]:
# ### SB001
# ## historical
# plan_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/online/streamflow/20750PG.csv"
# plan_data = pd.read_csv( plan_data_path, sep=",", header=0 )

# plan_data['datetime'] = pd.to_datetime(
#     [ t.astimezone(local_tz).strftime("%Y-%m-%d %H:%M:%S") for t in pd.to_datetime( plan_data['datetime'], format="%Y-%m-%dT%H:%M:%SZ%z" ) ]
# )
# plan_data.set_index( 'datetime', inplace=True )
# plan_data_cut = plan_data[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# plan_data_cut = resample_timeseries(df=plan_data_cut, offset=True, res_type='mean', step='1H')

# ## forecast
# forecast = pd.read_csv('/media/windows/projects/pp/SHARED/machine_learning/output/online/B001/SB001/R003/mean/forecast_1col.csv')
# forecast['index_CET'] = pd.to_datetime( forecast['index_CET'], format="%Y-%m-%d %H:%M:%S" )

# forecast.set_index('index_CET', inplace=True)
# forecast.index.name = 'datetime'
# forecast_cut = forecast[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# forecast_cut = resample_timeseries(df=forecast_cut, offset=True, res_type='mean', step='1H')

# forecast_cut['real'] = plan_data_cut['values']

# forecast_cut.plot(figsize=(15,10))

In [49]:
# ### SB005
# ## historical
# plan_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/online/streamflow/73950PG.csv"
# plan_data = pd.read_csv( plan_data_path, sep=",", header=0 )

# plan_data['datetime'] = pd.to_datetime(
#     [ t.astimezone(local_tz).strftime("%Y-%m-%d %H:%M:%S") for t in pd.to_datetime( plan_data['datetime'], format="%Y-%m-%dT%H:%M:%SZ%z" ) ]
# )
# plan_data.set_index( 'datetime', inplace=True )
# plan_data_cut = plan_data[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# plan_data_cut = resample_timeseries(df=plan_data_cut, offset=True, res_type='mean', step='1H')

# ## forecast
# forecast = pd.read_csv('/media/windows/projects/pp/SHARED/machine_learning/output/online/B001/SB005/R003/mean/forecast_1col.csv')
# forecast['index_CET'] = pd.to_datetime( forecast['index_CET'], format="%Y-%m-%d %H:%M:%S" )

# forecast.set_index('index_CET', inplace=True)
# forecast.index.name = 'datetime'
# forecast_cut = forecast[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# forecast_cut = resample_timeseries(df=forecast_cut, offset=True, res_type='mean', step='1H')

# forecast_cut['real'] = plan_data_cut['values']

# forecast_cut.plot(figsize=(15,10))

In [50]:
# ### SB007
# ## historical
# plan_data_path = "/media/lacie2022/data/meteo/eu/it/taa/aa/online/streamflow/37230PG.csv"
# plan_data = pd.read_csv( plan_data_path, sep=",", header=0 )

# plan_data['datetime'] = pd.to_datetime(
#     [ t.astimezone(local_tz).strftime("%Y-%m-%d %H:%M:%S") for t in pd.to_datetime( plan_data['datetime'], format="%Y-%m-%dT%H:%M:%SZ%z" ) ]
# )
# plan_data.set_index( 'datetime', inplace=True )
# plan_data_cut = plan_data[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# plan_data_cut = resample_timeseries(df=plan_data_cut, offset=True, res_type='mean', step='1H')

# ## forecast
# forecast = pd.read_csv('/media/windows/projects/pp/SHARED/machine_learning/output/online/B001/SB007/R003/mean/forecast_1col.csv')
# forecast['index_CET'] = pd.to_datetime( forecast['index_CET'], format="%Y-%m-%d %H:%M:%S" )

# forecast.set_index('index_CET', inplace=True)
# forecast.index.name = 'datetime'
# forecast_cut = forecast[ dt.datetime.strftime(start_date,"%Y-%m-%d %H:%M:%S"): dt.datetime.strftime(end_date,"%Y-%m-%d %H:%M:%S")]
# forecast_cut = resample_timeseries(df=forecast_cut, offset=True, res_type='mean', step='1H')

# forecast_cut['real'] = plan_data_cut['values']

# forecast_cut.plot(figsize=(15,10))

In [51]:
# vals_as_float = []
# for v in plan_data['values'].to_list():
#     try:
#         vals_as_float.append(float(v))
#     except:
#         vals_as_float.append(np.nan)

# plan_data.loc[:,'values'] = vals_as_float

# plan_data.resample('h').mean()

# plan_data = plan_data[ start_date:end_date ]

# plan_data.index = plan_data.index.strftime( "%Y-%m-%d %H:%M:%S" )

# c_output_path = "/home/daniele/documents/data/providers/it_taa_meteoaltoadige/merged/data/streamflow/"
# mkNestedDir(c_output_path)
# # sarentino_corvara_data.to_csv( c_output_path + start_date.strftime('%Y%m%d') + end_date.strftime('%Y%m%d') + ".csv" )
# plan_data.to_csv( c_output_path + "SB001.csv" )

In [52]:
# plan_data.plot(figsize=(15,10))

In [53]:
# checked_data = data_check( plan_data, style='streamflow' )

In [54]:
# checked_data.plot(figsize=(15,10))

In [55]:
# sarentino_data_path = "/media/windows/projects/pp/SHARED/alperia/streamflow/sarentino.csv"
# sarentino_data = pd.read_csv( sarentino_data_path, sep=",", skiprows=1, header=None )[[0,1,2]]

# sarentino_corvara_data = sarentino_data[[0,1]]
# sarentino_valdurna_data = sarentino_data[[0,2]]

In [56]:
# new_dt = []
# for r in sarentino_corvara_data[0].to_list():
#     date_time = r.split(" ")
#     day_month_year = date_time[0].split("/")
#     hour_secs = date_time[1].split(":")

#     new_dt.append( dt.datetime( 
#         int(day_month_year[2]), 
#         int(day_month_year[0]),
#         int(day_month_year[1]),
#         int(hour_secs[0]),
#         int(hour_secs[1]),
#         tzinfo=pytz.timezone("Europe/Rome")
#         )
#     )

# sarentino_corvara_data.loc[:,'datetime'] = new_dt
# sarentino_corvara_data.loc[:,'values'] = sarentino_corvara_data[1]
# sarentino_corvara_data = sarentino_corvara_data[['datetime','values']]
# sarentino_corvara_data.set_index('datetime', inplace=True)

# vals_as_float = []
# for v in sarentino_corvara_data['values'].to_list():
#     try:
#         vals_as_float.append(float(v))
#     except:
#         vals_as_float.append(np.nan)

# sarentino_corvara_data.loc[:,'values'] = vals_as_float

# sarentino_corvara_data.resample('h').mean()

# sarentino_corvara_data = sarentino_corvara_data[ start_date:end_date ]

# sarentino_corvara_data.index = sarentino_corvara_data.index.strftime( "%Y-%m-%d %H:%M:%S" )

# c_output_path = "/home/daniele/documents/data/providers/it_taa_meteoaltoadige/merged/data/streamflow/"
# mkNestedDir(c_output_path)
# # sarentino_corvara_data.to_csv( c_output_path + start_date.strftime('%Y%m%d') + end_date.strftime('%Y%m%d') + ".csv" )
# sarentino_corvara_data.to_csv( c_output_path + "SB002.csv" )

In [57]:
# sarentino_corvara_data.plot(figsize=(15,10))

In [58]:
# new_dt = []
# for r in sarentino_valdurna_data[0].to_list():
#     date_time = r.split(" ")
#     day_month_year = date_time[0].split("/")
#     hour_secs = date_time[1].split(":")

#     new_dt.append( dt.datetime( 
#         int(day_month_year[2]), 
#         int(day_month_year[0]),
#         int(day_month_year[1]),
#         int(hour_secs[0]),
#         int(hour_secs[1]),
#         tzinfo=pytz.timezone("Europe/Rome")
#         )
#     )

# sarentino_valdurna_data.loc[:,'datetime'] = new_dt
# sarentino_valdurna_data.loc[:,'values'] = sarentino_valdurna_data[2]
# sarentino_valdurna_data = sarentino_valdurna_data[['datetime','values']]
# sarentino_valdurna_data.set_index('datetime', inplace=True)

# vals_as_float = []
# for v in sarentino_valdurna_data['values'].to_list():
#     try:
#         vals_as_float.append(float(v))
#     except:
#         vals_as_float.append(np.nan)

# sarentino_valdurna_data.loc[:,'values'] = vals_as_float

# sarentino_valdurna_data.resample('h').mean()

# sarentino_valdurna_data = sarentino_valdurna_data[ start_date:end_date ]

# sarentino_valdurna_data.index = sarentino_valdurna_data.index.strftime( "%Y-%m-%d %H:%M:%S" )

# c_output_path = "/home/daniele/documents/data/providers/it_taa_meteoaltoadige/merged/data/streamflow/"
# mkNestedDir(c_output_path)
# # sarentino_valdurna_data.to_csv( c_output_path + start_date.strftime('%Y%m%d') + end_date.strftime('%Y%m%d') + ".csv" )
# sarentino_valdurna_data.to_csv( c_output_path + "SB003.csv" )

In [59]:
# sarentino_valdurna_data.plot(figsize=(15,10))

In [60]:
# enerpass_data_path = "/media/windows/projects/pp/SHARED/alperia/streamflow/enerpass.csv"
# enerpass_data = pd.read_csv( enerpass_data_path, sep="\t", skiprows=1, header=None )[[0,1]]

# new_dt = []
# for r in enerpass_data[0].to_list():
#     date_time = r.split(" ")
#     day_month_year = date_time[0].split("/")
#     hour_secs = date_time[1].split(":")

#     new_dt.append( dt.datetime( 
#         int(day_month_year[2]), 
#         int(day_month_year[0]),
#         int(day_month_year[1]),
#         int(hour_secs[0]),
#         int(hour_secs[1]),
#         tzinfo=pytz.timezone("Europe/Rome")
#         )
#     )

# enerpass_data.loc[:,'datetime'] = new_dt
# enerpass_data.loc[:,'values'] = enerpass_data[1]
# enerpass_data = enerpass_data[['datetime','values']]
# enerpass_data.set_index('datetime', inplace=True)

# enerpass_data.resample('h').apply(
#     lambda x: np.nan if x.isnull().sum() > 0 else x.mean())

# enerpass_data = enerpass_data[ start_date:end_date ]

# enerpass_data.index = enerpass_data.index.strftime( "%Y-%m-%d %H:%M:%S" )

# c_output_path = "/home/daniele/documents/data/providers/it_taa_meteoaltoadige/merged/data/streamflow/"
# mkNestedDir(c_output_path)
# # enerpass_data.to_csv( c_output_path + start_date.strftime('%Y%m%d') + end_date.strftime('%Y%m%d') + ".csv" )
# enerpass_data.to_csv( c_output_path + "SB004.csv" )

In [61]:
# enerpass_data.plot(figsize=(10,15))

In [62]:
# enerpass_data.dropna()